In [1]:
from datetime import datetime
from packaging import version

import os
import tensorflow as tf
import tensorflow_datasets as tfds


In [2]:
tfds.disable_progress_bar()
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(normalize_img)
ds_train = ds_train.batch(32)

ds_test = ds_test.map(normalize_img)
ds_test = ds_test.batch(32)


In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
  tf.keras.layers.Dense(128,activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy']
)


In [4]:
# Create a TensorBoard callback
logs = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")

tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')

model.fit(ds_train,
          epochs=2,
          validation_data=ds_test,
          callbacks = [tboard_callback])


Epoch 1/2
1875/1875 [==============================] - 5s 2ms/step - loss: 0.2619 - accuracy: 0.9254 - val_loss: 0.1422 - val_accuracy: 0.9579
Epoch 2/2
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1177 - accuracy: 0.9654 - val_loss: 0.1064 - val_accuracy: 0.9675
